# Efficient vector similarity search 

### Enhancing search results with vector embeddings and Redis

Import the necessary libraries 

Define a color class for later use:


In [1]:
import random
import numpy as np
import pandas as pd
import time
from redis import Redis
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
from redis.commands.search.result import Result

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [5]:
all_columns_df = pd.read_csv("data/translated.csv")
all_columns_df.reset_index(drop=True, inplace=True)   

In [2]:
import openai

api_key = "API_KEY"

def set_openai_key(api_key):
    openai.api_key = api_key

set_openai_key(api_key)

In [3]:
def gpt3_algorithm_v2(text):
  # Completion function call engine: text-davinci-002

    Platformresponse = openai.Completion.create(
        engine="text-davinci-002",
        prompt=f"Identify and list the keywords in this tweet:{text}.",
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        )

    return Platformresponse.choices[0].text

In [8]:
#Loop through dataframe
for index, row in all_columns_df.iterrows():
    #extract column to be translated
    text_to_translate = all_columns_df.loc[index, 'translated_text']
    #translate using OpenAI API
    translated_text = gpt3_algorithm_v2(text_to_translate)
    #add translated text to new column
    all_columns_df.at[index, 'keywords'] = translated_text
    if index % 100 == 0:
        print(index)
#return dataframe with original and translated columns
    all_columns_df = all_columns_df

0
100
200
300
400
500
600
700
800
900


In [7]:
all_columns_df.head()

,conversation_id,created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/27/screen_name,user_mentions/28/id_str,user_mentions/28/name,user_mentions/28/screen_name,user_mentions/29/id_str,user_mentions/29/name,user_mentions/29/screen_name,primary_key,translated_text,keywords
0,1639413359123505152,2023-03-24T23:46:18.000Z,1,"Voilà pourquoi ils nous polluent cet espace, a...",RwandaIsKilling,Nek,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-24T23:46:18.000Z-This is why they poll...,This is why they pollute this space with their...,"\n\nRwanda, pollute, space, nonsense, analyses"
1,1537820130230534152,2022-06-26T03:55:57.000Z,0,@kamikriss #DonotVisitRwanda #Rwandaisaterrori...,DonotVisitRwanda,Rwandaisaterroriststate,InvestigatetheDRCgenocide,OpentheDRCGenocidecourt,RwandaIsKillingInDRC,Rwandaiskilling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-26T03:55:57.000Z-Do not visit Rwanda. ...,Do not visit Rwanda. Rwanda is a terrorist sta...,\n\nDo not visit Rwanda. Rwanda is a terrorist...
2,1628392315747270658,2023-02-22T13:52:36.000Z,2,"N'OUBLIE PAS, LE #Rwanda TUE EN #RDCongo \nDon...",Rwanda,RDCongo,Rwanda,DRC,RwandaIsKillingInDRC,KagameMurderer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2023-02-22T13:52:36.000Z-Don't forget, #Rwanda...","Don't forget, #Rwanda is killing in the #DRC.\...","\n\n#Rwanda, #DRC, #RwandaIsKillingInDRC, #Kag..."
3,1538788031687274498,2022-06-22T18:25:42.000Z,1,@Claudbam @Kazungukirimvi @CSitault @RDBrwanda...,DonotVisitRwanda,Rwandaisaterroriststate,InvestigatetheDRCgenocide,OpentheDRCGenocidecourt,RwandaIsKillingInDRC,Rwandaiskilling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-22T18:25:42.000Z-@Claudbam @Kazungukir...,@Claudbam @Kazungukirimvi @CSitault @RDBrwanda...,\n\n@Claudbam @Kazungukirimvi @CSitault @RDBrw...
4,1614914990947074049,2023-01-16T09:18:32.000Z,0,#DRCAgainstPeace,DRCAgainstPeace,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-16T09:18:32.000Z-#DRCAgainstPeace,#DRCAgainstPeace,"\n\nDRC, peace"


In [32]:
all_columns_df['keywords'] = all_columns_df['keywords'].str.replace('/n/n', '')

In [335]:
all_columns_df = pd.read_csv("data/translated.csv")
all_columns_df.reset_index(drop=True, inplace=True)   
# all_columns_df.to_csv('data/translated.csv', index=False)

In [190]:
# for i in all_columns_df:
#     all_columns_df[i] = all_columns_df[i].astype(str)
all_columns_df.replace(np.nan, '',inplace=True)

In [336]:
all_columns_df = all_columns_df[['conversation_id', 'favorite_count', 'full_text', 'is_quote_tweet', 'is_retweet', 'primary_key', 'keywords']]
all_columns_df = all_columns_df.astype(str)

In [337]:
all_columns_df[['is_quote_tweet', 'is_retweet']] = all_columns_df[['is_quote_tweet', 'is_retweet']].replace('True', 'Yes')
all_columns_df[['is_quote_tweet', 'is_retweet']] = all_columns_df[['is_quote_tweet', 'is_retweet']].replace('False', 'No')
all_columns_df[['is_quote_tweet', 'is_retweet']] = all_columns_df[['is_quote_tweet', 'is_retweet']].replace('nan', 'No')

In [338]:
all_columns_df[['is_quote_tweet', 'is_retweet']].head()

,is_quote_tweet,is_retweet
0,Yes,No
1,No,No
2,Yes,No
3,No,No
4,Yes,No


In [339]:
tweet_metadata = all_columns_df.to_dict(orient='index')
#str(tweet_metadata)

In [70]:
redis_conn = Redis(
      host=os.environ["REDIS_HOST"],
      port=os.environ["REDIS_PORT"],
      password=os.environ["REDIS_PASSWORD"]))
redis_conn
print ('Connected to redis')

Connected to redis


SentenceTransformer

 a pre-trained Sentence Transformer model called distilroberta-v1

In [385]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [386]:
%%time

keywords =  [tweet_metadata[i]['keywords']  for i in tweet_metadata.keys()]
keywords_vectors = [ model.encode(sentence) for sentence in keywords]

CPU times: total: 17 s
Wall time: 22.7 s


Check the dimensions of the embeddings

In [358]:
redis_conn.hgetall('Tweet:164:DRC, Against, Peace, Failed, State')

{b'conversation_id': b'1618691899258445825',
 b'favorite_count': b'0',
 b'is_retweet': b'No',
 b'keyword_vector': b'\xc8\xaf^\xbdk_\r\xbc\x90z\x16=U\x95s<\x87\x1f\xad\xbc\x05\x1d\xfd;\xa9\xfe1\xbb|=\x8a=Q\x17\x0c<4\xe9\x92\xbboc\xff\xbc.\xb8d=\xfaQ\xcb\xbc[o\xe2\xbcY\xd7\x1c;\x9ek\x8f;#9\x8d=\xe16\x07=w\xac\xa7\xbb\x16\xc4\xe1;\xfb\xe0\x00\xbd/?R\xbc\xe8Q%\xbdD\x0c@\xbdFAu\xbd\x0b\xf9\xb0\xbb\x13\x91-\xbd\xba"?\xbc\xb8s\xd2\xbd\xf6\xce\xdd</T!<\x88F\xcb;*\xb1\xcb<\xb1P5\xbc\xb7\\\x9b<\xdd\x9ek\xba\x12\x95\xc0\xbc\x0c)\x80\xbb\t\xf2-;\x8d\xd2\x80<:n\x01\xbd\xd4-N<\xeep\xd4\xbcS\'K=\xbb\xeb\xda\xbc\xa9m+=EV\xa5\xbd\x82\xa6d\xbde\xa4\x86\xbc#\xb6\x01\xbd\xb2$\xe1<\xc4\xff":\xe5\x8c\x10\xbd\x8a\x866\xbd\xafv\x99=I(b=\x16&+<90\x01<\x84\xc5G\xbb\xb0\xdc\x1f=(\xff@=,\xd4\xfa\xbc\x14%\x85\xbd_m\x0e\xbd\xd8\xc0\xc2\xbaZud=\xa6(\xa2\xbc\xa8\x19\xb1<?\x88i<:V\xb1\xbb!I@\xbd2)Z\xbb`\x00V=\x1am:\xbdD!\xe1\xbaL-k<\xb8\xa0w\xbd\xa2\xb42\xbdr\xf8H\xbd\x87\xe9\x00\xbc\n\xaf\x8d\xba\x1a\xf6<=|\xc1\x16=\

In [42]:
len(keywords_vectors)

1000

In [45]:
len(tweet_metadata[0])

399

In [258]:
# Check one of the products
tweet_metadata[0]

{'conversation_id': '1639413359123505152',
 'favorite_count': '1',
 'full_text': 'Voilà pourquoi ils nous polluent cet espace, avec des analyses à la mords-moi-le-nœud ! #RwandaIsKilling #Nek',
 'is_quote_tweet': 'Yes',
 'is_retweet': 'No',
 'primary_key': '2023-03-24T23:46:18.000Z-This is why they pollute this space with their nonsense analyses! #RwandaIsKilling #Nek',
 'keywords': 'Rwanda, pollute, space, nonsense, analyses',
 'keyword_vector': b'7@\xa0:\x96\xf9\xbf\xbc+|E9\x18"\xdf\xbc\x85c\'\xbc\xb3\xfe\xe1\xbc\';\xb0\xbc\x05,==z\x0f?=\x1d\x87?\xbdm\x96\xaa\xbd\x85f\'\xbd\x11\'9<\xffBo\xbd\xc1\xf3\x7f<e\x88b\xbd\xbd\x02\xaf<|\x9e4<[\xe9\x11\xbd\x8e\xf7\xb9<1q\x08=\xc9\xa5\xbf=\xe4\xb5\xd6\xbc\xed\xcf\xdd\xbc{\xbb\xa7=\xdd\xea\xb7<X\x04m\xbd\x8c\xfcI\xbd4@\x81\xbb3\x84\xeb\xbcMg\xaa<f\x18|\xbc\xae9\x10=\r\x15K\xbd\x9c\x9c\x89\xbc\xed\x13\x0f<59\x15;\x08\xe4I\xbd\x8fo\x81=\xf8\r\xc8\xbc\x8d\x83\xb9=\xfe\xd2\x16=\x150\x04\xbcs\x84\xd9<\x0c\xe3\xc2<\xf9+\xe2\xbb\xa2/\xa1\xbcq\xc6N\xbd\

In [406]:
redis_conn.flushall()


True

Now we have 1000 embeddings for the 1000 tweets

Prepare 3 utility functions

One for loading the product data and two for creating the index on Vector fields

In [393]:
def load_vectors(client:Redis, tweet_metadata, vector_dict, vector_field_name):
    p = client.pipeline(transaction=False)
    for index in tweet_metadata.keys():    
        #hash key
        key='Tweet:'+ str(index)+ ':' + tweet_metadata[index]['keywords']
        #hash values
        tweets_metadata = tweet_metadata[index]
        keywords_vector = vector_dict[index].astype(np.float32).tobytes()
        tweets_metadata[vector_field_name]=keywords_vector
        # HSET
        p.hset(key, mapping=tweets_metadata)

    p.execute()

def create_flat_index (redis_conn,vector_field_name,number_of_vectors, vector_dimensions=512, distance_metric='L2'):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "FLAT", {"TYPE": "FLOAT32", "DIM": vector_dimensions, "DISTANCE_METRIC": distance_metric, "INITIAL_CAP": number_of_vectors, "BLOCK_SIZE":number_of_vectors }),
        TagField("keywords"),
        # TextField("is_quote_tweet"),
        # TextField("primary_key"),
        # TextField("conversation_id"),
        # TextField("is_retweet")        
    ])


def create_hnsw_index (redis_conn,vector_field_name,number_of_vectors, vector_dimensions=512, distance_metric='L2',M=40,EF=200):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "HNSW", {"TYPE": "FLOAT32", "DIM": vector_dimensions, "DISTANCE_METRIC": distance_metric, "INITIAL_CAP": number_of_vectors, "M": M, "EF_CONSTRUCTION": EF}),
        TagField("keywords"),
        # TextField("is_quote_tweet"),
        # TextField("primary_key"),
        # TextField("conversation_id"),
        # TextField("is_retweet")   
    ])    

# Comparison of flat and HNSW indexing for approximate nearest neighbor search

Index & Query the new data

In [394]:
%%time

KEYWORDS_EMBEDDING_FIELD='keyword_vector'
TEXT_EMBEDDING_DIMENSION=784
NUMBER_TWEETS=1000

print ('Loading and Indexing + ' +  str(NUMBER_TWEETS) + ' tweets')

#flush all data
redis_conn.flushall()

#create flat index & load vectors
create_flat_index(redis_conn, KEYWORDS_EMBEDDING_FIELD,NUMBER_TWEETS,TEXT_EMBEDDING_DIMENSION,'COSINE')
load_vectors(redis_conn,tweet_metadata,keywords_vectors,KEYWORDS_EMBEDDING_FIELD)

Loading and Indexing + 1000 tweets
CPU times: total: 109 ms
Wall time: 1.59 s


Query our FLAT index

In [395]:
%%time
topK=5
tweet_query='rwanda drc'
#product_query='cool way to pimp up my cell'

#vectorize the query
query_vector = model.encode(tweet_query).astype(np.float32).tobytes()

#prepare the query
q = Query(f'*=>[KNN {topK} @{KEYWORDS_EMBEDDING_FIELD} $vec_param AS vector_score]').sort_by('vector_score').paging(0,topK).return_fields('vector_score','keywords_vectors').dialect(2)
params_dict = {"vec_param": query_vector}


#Execute the query
results = redis_conn.ft().search(q, query_params = params_dict)

#Print similar products found
for tweet in results.docs:
    print ('***************Similar tweets  found ************')
    print (color.YELLOW + 'Score = ' +  color.END  + tweet.vector_score)
   # print (color.YELLOW + 'conversation_id = ' +  color.END  + tweet.conversation_id)
    print (color.BOLD + 'keywords = ' +  color.END + tweet.keywords)
    #print (color.YELLOW + 'is_quote_tweet = ' +  color.END  + tweet.is_quote_tweet)
   # print (color.YELLOW + 'primary_key = ' +  color.END  + tweet.primary_key)

CPU times: total: 46.9 ms
Wall time: 87.8 ms


In [400]:
print(query_vector)

b'_\xa3\x03=\xf67\x19\xbd\x17\xef\x11\xbc-\x14"\xbd\tA\x92=\x046\xb6\xbd\xec\xefN\xbb?@\x87=\xf7=\t=\xbd\xad\xdb\xbcK\xfe\x10\xbd\xbf\xbba=]\xa8\x80\xbcDr;\xbd\xd8ey<L/\xae\xbc\xc7 \x18=q70\xbd8\xf8\xcc\xbb\xe1\x05B<\x02W\xc1<^\xa3\n=U\xc2\x0e\xbd\x86i\xa5<\xaf!\x83=\x16_\xac;\xcfTC\xbc\xfa\xf6\x17\xbb\xcf\xf3\n\xbd\x14\xfa\xdd=\x9fm\xf0<\x87\x9f}\xbd\t7\xb6<\xe4\xb7\x85<\x16\x7fD\xbd\x9e;!<e\x00u\xb9\x84\xd0\xd6\xbc\x01]\xc1<\xd9\x9c\xa3<}a\xeb<pP\x86\xbd\x1eKZ=\xe5\xbe\x82\xbc%L|\xbc\xd2\x96\x91<o"~\xbc\xd75\xeb\xbc &7\xbc\xc5&\xf0\xbc6@f\xbb\x17\x16\xee<_T\xde\xbcn.\x8b\xbc\x8cX\xc0\xbc\x88!W;7\xa6\xf1<\x8afp=\xe8\x91\x94=g\xe8\x84\xbc\x1e\xeb\x06<\xad\xa1\xae<\xa4c\x0b=\xee2\x18=\x98\xf4\xa7=\xed/\x12=+6\xa2<n\xee\x11<\x8e\x13V:\xb2\xe5\x19\xbd"\xe8\x85<\xe6!Y\xbd]G\x05=\x8a\x98\x9b<\xd0\x15f<\xa4\xd5\xd8\xbca!\x95\xbc\x7f\xea\xf5\xbc\xa2u\xa3\xbd\xfc[\xed\xbc(0\x81=\x03u!\xbb\x9b\xa5$\xbcH\xd8\x0c<\x94\xa5\'=\x85\xdbu<LT\xcc<\x00J\xb5=\xb4i\x8a\xbc8\xae\xba\xbb\xa3,$\xbc7(\xf5\xbb

Query our HNSW index

In [401]:
%%time
KEYWORDS_EMBEDDING_FIELD='keyword_vector'
TEXT_EMBEDDING_DIMENSION=784
NUMBER_TWEETS=868

print ('Loading and Indexing + ' +  str(NUMBER_TWEETS) + ' tweets')

#flush all data
redis_conn.flushall()

#create flat index & load vectors
create_hnsw_index(redis_conn, KEYWORDS_EMBEDDING_FIELD,NUMBER_TWEETS,TEXT_EMBEDDING_DIMENSION,'COSINE',M=40,EF=200)
load_vectors(redis_conn,tweet_metadata,keywords_vectors,KEYWORDS_EMBEDDING_FIELD)

Loading and Indexing + 868 tweets
CPU times: total: 203 ms
Wall time: 1.8 s


In [397]:

topK=5
tweet_query='rwandaiskilling'
#product_query='cool way to pimp up my cell'

#vectorize the query
query_vector = model.encode(tweet_query).astype(np.float32).tobytes()

#prepare the query
q = Query(f'*=>[KNN {topK} @{KEYWORDS_EMBEDDING_FIELD} $vec_param AS vector_score]').sort_by('vector_score').paging(0,topK).return_fields('vector_score').dialect(2)
params_dict = {"vec_param": query_vector}




In [398]:

#Execute the query
results = redis_conn.ft().search(q, query_params = params_dict)



In [399]:
print(results)

Result{0 total, docs: []}


In [ ]:
#Print similar products found
for tweet in results.docs:
    print ('***************Product  found ************')
    print (color.YELLOW + 'Score = ' +  color.END  + tweet.vector_score)